In [568]:
# exploring data attached 
import pandas as pd 


In [569]:
jobs = pd.read_excel("GA_task.xlsx")
jobs

,1,Unnamed: 1,2,Unnamed: 3,3,Unnamed: 5,4,Unnamed: 7,5,Unnamed: 9,...,46,Unnamed: 91,47,Unnamed: 93,48,Unnamed: 95,49,Unnamed: 97,50,Unnamed: 99
0,R,T,R,T,R,T,R,T,R,T,...,R,T,R,T,R,T,R,T,R,T
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18
5,5,21,6,27,4,17,7,27,10,42,...,7,50,4,26,4,19,1,20,2,21
6,3,19,8,33,10,31,2,43,1,29,...,10,29,8,17,3,38,10,21,2,33
7,2,18,2,49,8,50,7,33,4,21,...,10,33,7,12,6,24,5,25,8,46
8,7,45,9,39,7,34,1,10,9,35,...,10,11,6,42,8,20,4,18,3,40
9,6,10,1,15,5,38,1,35,7,43,...,5,13,2,24,4,48,6,40,7,27


So to not get to confused with vocabulary. We face following problem we have 50 `tasks` each of them requires Resource R for some period of time T. Let's call one single element of this Task a Job for each Task Jobs need to be performed within declared order. Each resource we'll call a Machine so for job to be done we need some working time on specified machine.

In [570]:

class Task: 
    def __init__(self , jobs: list , id:int):
        """
        Represents a single task that requries some jobs within order to be done 
        Jobs a list of pair R - T for this task 
        """
        self.jobs = jobs                 # Jobs in order to be performed
        self.id = id            # as a Id in excel file is 1 -index and python is  0 - index
        self.current_timestamp = 0     # in our timeline kinda time mark at which we're with task so for example if we start first job at time 0 and 15 T is required we set this to 15.                        
    def __str__(self) -> str:
        return f"Task id: {self.id} \n jobs to perform: \n {self.jobs}"
    
    def return_machines(self):
        machines_required = []
        for job in self.jobs:
            machines_required.append(job[0])
        return machines_required
    
    def get_timestamp(self):
        return self.current_timestamp
    
    def get_machine_ix(self):
        return  self.jobs[0][0] - 1       # first job -> resource (1 index) - 1 to get 0_ix 
    
    def get_T(self):
        return  self.jobs[0][1]
    
    def next_step(self,time):              # we update our task after scheduling of first job to do
        self.jobs.pop(0)
        self.current_timestamp = time
    
   
        

In [571]:
class Machine:
    """Initialize machine object """
    def __init__(self, ix):
        self.id = ix   
        self.busy_until = 0   
        self.schedule =  []  # for now let's separete self.tasks for plot or even omit and create another parameter with let's call it occupation slots start-end of job performed on machine
        self.tasks = [[],[],[],[]]  # we'll try to perform visualization in a gant chart style, probably will work only on some small subset of our tasks 
        # format to then convert it to dataframe and utilize plotly so [[machine_id],[task_id],[start_time],[end_time]
        self.schedule.append((0,0)) # for consistency let's assume that on start machine is occupied with start at 0 and end at 0
        self.task_scheduled = 1  # we now that at init task list has one trivial task
    def get_timestamp(self):
        return self.busy_until
        
    # let's also here add method to give a machine abilty to return slot at which it can perform given task, 
    def schedule_task(self,timestamp,time):
        """Timestamp first point in time at which task can be scheduled, time -> for task big T - time to perform job"""
        # we need to keep schedule in order 
        if self.task_scheduled == 1:
            self.schedule.append((timestamp,timestamp+time))
            self.busy_until = timestamp+time
            self.tasks[2].append(timestamp)                        # getting our table for chart ready probably to improve
            self.tasks[3].append(timestamp+time)
        else:
            for i in range(1,self.task_scheduled):
                end_of_previous_job = self.schedule[i-1][1]
                start_of_next_job = self.schedule[i][0]
                if ((timestamp <= end_of_previous_job) and (end_of_previous_job+time <=start_of_next_job)):   # checking two conditions to answer question Can we fit new task somewhere between old scheduled tasks
                    self.schedule.insert(i,(end_of_previous_job,end_of_previous_job+time))   # put task into gap
                    self.tasks[2].append(end_of_previous_job)
                    self.tasks[3].append(end_of_previous_job+time)
                    # only situation that end of current job in the task will differ from machine.busy_unitl
                    self.tasks[0].append(self.id)
                    # print(timestamp+time)
                    return end_of_previous_job+time # we need this to update task timeline 
                # polish cuz tired -> tu opisaliśmy sytuację gdzie możemy zacząć zadanie w trakcie kiedy jeszcze wyonujemy coś na maszynie, ale zmiescimy zadanie przed kolejnym już zaplanowanym 
                # mozemy jeszcze mieć sytuację, ze zaczniemy zadanie w trakcie przerwy, ale będizemy mogli rozpocząć dopiero po jakimś czasie od zwolnienia maszyny, ale i tak się wyrabiamy przed kolejnym zaplanowanym
                elif((end_of_previous_job <= timestamp <= start_of_next_job) and  (timestamp+time <=start_of_next_job)):
                    self.schedule.insert(i,(timestamp,timestamp+time))   # put task into gap
                    self.tasks[2].append(timestamp)
                    self.tasks[3].append(timestamp+time)
                    # only situation that end of current job in the task will differ from machine.busy_unitl
                    self.tasks[0].append(self.id)
                    # print(timestamp+time)
                    return timestamp+time # we need this to update task timeline 
                
                
                    
            else:
                # we need to append at the end of task list for the machine but also first check if task.timepoint (end of previous job) > machine.timepoint(end of last scheduled task)
                if (timestamp<self.busy_until):
                    self.schedule.append((self.busy_until,self.busy_until+time))
                    self.busy_until  += time
                    self.tasks[2].append(self.busy_until)
                    self.tasks[3].append(self.busy_until+time)
                else:
                    self.schedule.append((timestamp,timestamp+time))
                    self.busy_until = timestamp + time
                    self.tasks[2].append(timestamp)
                    self.tasks[3].append(timestamp+time)
        
        self.tasks[0].append(self.id)  # adding machine index just for getting good dataframe for plot  
        
        # print(self.busy_until)
        return self.busy_until # in every other situation we can return self.busy_until

    def update_task_list(self,task_id):
        # during step of adding we also add task_id to the table:
        self.tasks[1].append(task_id)
        
    def df_task_list(self):
        return pd.DataFrame({'machine_id':self.tasks[0],'task_id':self.tasks[1],'start_time':self.tasks[2],'end_time':self.tasks[3]})
    
    def next_step(self):
        self.task_scheduled += 1
        
        
        

In [572]:

def read_tasks(n = None):
    jobs = pd.read_excel("GA_task.xlsx")
    table = []
    col_number = len(jobs.columns)
  
    if n:
        length = 0
        for i in range(0, col_number,2):
            id = jobs.columns[i]
            task_jobs = list(zip(jobs.iloc[1:,i],jobs.iloc[1:,i+1]))
            table.append(Task(jobs=task_jobs,id = id)) # we add zipped pairs of machine and time for each task , and we convert to 0 index
            length +=1
            if length == n: return table
        
    for i in range(0, col_number,2):
        id = jobs.columns[i]
        task_jobs = list(zip(jobs.iloc[1:,i],jobs.iloc[1:,i+1]))
        table.append(Task(jobs=task_jobs,id = id)) # we add zipped pairs of machine and time for each task , and we convert to 0 index
    
    return table


def extract_machines(task_table: list):
    """ Requires task_table -> list of Task objects"""
    list_of_machines = []
    for task in task_table:
        machines_for_task = task.return_machines()   # this method returns list of machines for this task
        for machine in machines_for_task:
            list_of_machines.append(machine)   # we append machine requried
    # in the end remove duplicates and return 
    return list(set(list_of_machines))
    

In [573]:
task_table = read_tasks(5)

In [574]:
task_table

In [575]:
machines = extract_machines(task_table)

In [576]:
# now we are ready to initialize table of machines 

machine_table = []

for ix in machines:
    machine_table.append(Machine(ix = ix))

In [577]:
machine_table

In [578]:
machine_table[0].tasks[2]

[]

In [579]:
# and now finally Genotype here we'll probably make the most of calcualtion and also for each genotype we need to calcualte time to complete tasks in given order 
# so to do that we'll utilize Machine and Task classes and their methods
import random
class Genotype:
    def __init__(self,task_table,machine_table):
        """ It needs to be initialized with tables of Tasks and Machines"""
        # with assumption that each task requires same number of jobs length of genotype will be len(task_table) * len(machine_table)
        self.geno = self.initialize_geno(len(task_table),len(machine_table))
        self.task_table = task_table
        self.machine_table = machine_table
        self.time = 0 
    def initialize_geno(self,n,k):
        geno = list(range(0,n)) * k     # just creating random genotype 
        random.shuffle(geno)
        return geno
        
    def calculate_time(self): 
        for task_ix in self.geno:
            machine_ix = self.task_table[task_ix].get_machine_ix()  # extracting first machine_x from task
            task_timestamp = self.task_table[task_ix].get_timestamp()   # extracting current position in time of this task end
            # machine_timestamp = self.machine_table[machine_ix].get_timestamp()  # extracting current position in time of machine required
            # # two options
            time = self.task_table[task_ix].get_T() # getting Time of job
            
            task_till = self.machine_table[machine_ix].schedule_task(task_timestamp,time)   # this function will return point in time at which we're with this task
            # just for future visualization
            self.machine_table[machine_ix].update_task_list(task_ix)
            # and now we need to pop this job from task requirments It's done or at least planned to be done
            self.task_table[task_ix].next_step(task_till)   # we remove first job from job list for this task and we update point in time of this task
            
            self.machine_table[machine_ix].next_step()   # just keep tracking of number scheduled jobs for machine
             
        # at the end of this loop we have all machines in table scheduled we now we can extract time at which each of them finish and max of it is time in this genotype
        
        max = 0
        for machine in self.machine_table:
            if (machine.busy_until > max):
                max = machine.busy_until
        self.time = max
            
            
            
        
        
        
        
        


In [580]:
testowy_genotyp = Genotype(task_table,machine_table)
testowy_genotyp.geno

[4,
 3,
 2,
 2,
 0,
 1,
 4,
 4,
 1,
 3,
 4,
 4,
 2,
 1,
 0,
 1,
 0,
 4,
 0,
 0,
 3,
 2,
 1,
 3,
 0,
 1,
 3,
 2,
 1,
 2,
 3,
 0,
 3,
 0,
 3,
 4,
 3,
 4,
 0,
 1,
 2,
 4,
 3,
 1,
 1,
 2,
 0,
 2,
 2,
 4]

In [581]:
for i in machine_table:
    print(i.schedule)

[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]


In [582]:
for i in machine_table:
    print(i.tasks)

[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]


In [590]:
df_list = []
for i in machine_table:
    df_list.append(i.df_task_list())

In [591]:
df_sumup = pd.concat(df_list,axis=0)
df_sumup

,machine_id,task_id,start_time,end_time
0,1,2,52,70
1,1,0,85,132
2,1,4,151,180
3,1,3,296,306
4,1,3,306,341
5,1,1,275,290
6,1,2,471,500
7,1,4,384,396
0,2,4,89,109
1,2,1,171,220


In [592]:
df_sumup[df_sumup['machine_id']==1].sort_values('start_time',ascending=True)

,machine_id,task_id,start_time,end_time
0,1,2,52,70
1,1,0,85,132
2,1,4,151,180
5,1,1,275,290
3,1,3,296,306
4,1,3,306,341
7,1,4,384,396
6,1,2,471,500


In [593]:
df_sumup[df_sumup['task_id']==1].sort_values('start_time',ascending=True)

,machine_id,task_id,start_time,end_time
1,5,1,41,58
0,10,1,41,54
1,4,1,82,111
1,7,1,82,105
1,6,1,111,138
1,8,1,138,171
1,2,1,171,220
5,1,1,275,290
4,9,1,275,314
5,9,1,290,302


In [589]:
for i in task_table:
    print(i)

Task id: 1 
 jobs to perform: 
 [(5, 46)]
Task id: 2 
 jobs to perform: 
 [(7, 13)]
Task id: 3 
 jobs to perform: 
 [(1, 26)]
Task id: 4 
 jobs to perform: 
 [(9, 39)]
Task id: 5 
 jobs to perform: 
 [(7, 36)]


In [557]:
for i in task_table:
    print(i.return_machines())

[5]
[7]
[1]
[9]
[7]


In [587]:
testowy_genotyp.calculate_time()

In [588]:
testowy_genotyp.time

500

In [559]:
for i in machine_table:
    print(i.df_task_list())

   machine_id  task_id  start_time  end_time
0           1        2          52        70
1           1        0          85       132
2           1        4         192       221
3           1        3         400       410
4           1        3         410       445
5           1        2         272       301
6           1        1         342       357
7           1        4         367       379
   machine_id  task_id  start_time  end_time
0           2        4          89       109
1           2        0         236       254
2           2        1         303       352
3           2        3         346       389
   machine_id  task_id  start_time  end_time
0           3        0          36        85
1           3        0         217       236
   machine_id  task_id  start_time  end_time
0           4        2          24        52
1           4        2         102       119
2           4        1         148       177
3           4        4         221       242
   machine

In [560]:
df_sumup

,machine_id,task_id,start_time,end_time


In [526]:
df_sumup['end'] = df_sumup['end_time'] - df_sumup['start_time']

In [404]:
from plotly.express import bar

In [464]:
bar(data_frame=df_sumup,base='start_time',x = 'end',y = 'machine_id',color = "task_id",orientation='h')

In [18]:
for i in task_table:
    print(i)

Task id: 1 
 jobs to perform: 
 [(9, 22), (3, 49), (1, 47), (9, 30), (5, 21), (3, 19), (2, 18), (7, 45), (6, 10), (8, 39), (5, 46)]
Task id: 2 
 jobs to perform: 
 [(5, 17), (10, 13), (7, 23), (4, 29), (6, 27), (8, 33), (2, 49), (9, 39), (1, 15), (9, 12), (7, 13)]
Task id: 3 
 jobs to perform: 
 [(5, 24), (4, 28), (1, 18), (5, 32), (4, 17), (10, 31), (8, 50), (7, 34), (5, 38), (1, 29), (1, 26)]
Task id: 4 
 jobs to perform: 
 [(8, 40), (5, 47), (5, 47), (7, 18), (7, 27), (2, 43), (7, 33), (1, 10), (1, 35), (7, 15), (9, 39)]
Task id: 5 
 jobs to perform: 
 [(9, 14), (7, 45), (6, 30), (2, 20), (10, 42), (1, 29), (4, 21), (9, 35), (7, 43), (1, 12), (7, 36)]


In [19]:
class Machines:
    """Initialize machine object """
    def __init__(self, ix):
        self.id = ix + 1   # machines are also 0 index
        self.busy_until = None   
        self.tasks = [[],[],[],[]]  # we'll try to perform visualization in a gant chart style, probably will work only on some small subset of our tasks 
        # format to then convert it to dataframe and utilize plotly so [[machine_id],[task_id],[start_time],[end_time]
        self.machines_tasks = self.assign_tasks()
        
    
    def assign_tasks(self):
        table = []
        for i in self.tasks:  # just small loop to figure out of numbers of machines thats required
            for j in i:
                table.append(j[0])
        machines_number = len(set(table))
        
        
        machines = [[] for _ in range(machines_number)]   # 'machines' ready now fill them with tasks to perform on  them # we have 0 index there 
        for task, task_req in enumerate(self.tasks):
            for job in task_req:
                machines[job[0]].append(task)    # in the table with machines we store task which we'll be performed
        return machines
 